In [1]:
import sys
import os
import torch
from accelerate.test_utils.testing import get_backend
import pandas as pd
import json
import numpy as np
import re
import nltk
import time
from sentence_transformers import SentenceTransformer
import jsonlines
from scipy.spatial.distance import cosine
import pandas as pd
from collections import Counter
import textwrap

# Add the folder to sys.path
sys.path.append(os.path.abspath("prompt engineering"))

# Import your module
from eval import *
from gen import *
from plot_func import *


In [4]:
# check token
access_token = os.getenv("HF_TOKEN")

In [3]:
# check gpu
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [4]:
DEVICE, _, _ = get_backend()

____________________________
### Masking named entities

In [15]:
import os
import nltk
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer

import logging
from transformers.utils import logging as hf_logging

# Set Transformers' logging level to ERROR
hf_logging.set_verbosity_error()

# Optionally suppress all warnings from Python if needed
logging.getLogger().setLevel(logging.ERROR)


def process_and_label_sentences_m(input_path, label_embeddings_path, output_path, model_name="all-MiniLM-L6-v2", per_sentence=True, mask_entities=True):
    """
    Processes responses into embeddings, matches them to labels,
    and writes data to a .jsonl file.

    Args:
        input_path (str): Path to input JSONL with "response" fields.
        label_embeddings_path (str): Path to label embeddings JSONL.
        output_path (str): Output JSONL file path.
        model_name (str): SentenceTransformer model to use.
        per_sentence (bool): If True, embed each sentence. Otherwise, embed the full response.
        mask_entities (bool): If True, mask named entities before embedding.
    """
    # Load embedding model
    model = SentenceTransformer(model_name)

    # Load NER model if masking is enabled
    ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple") if mask_entities else None

    # Load data
    data = read_jsonl(input_path)
    label_entries = read_jsonl(label_embeddings_path)

    label_emb = np.array([np.array(e['embedding'], dtype=np.float32) for e in label_entries])
    labels = [e['label'] for e in label_entries]

    for item in data:
        response_text = item.get("response", "")
        units = nltk.sent_tokenize(response_text) if per_sentence else [response_text]

        masked_units = []
        for sentence in units:
            if mask_entities and ner:
                entities = ner(sentence)
                masked_sentence = sentence
                for ent in entities:
                    if ent["entity_group"] == "LOC":
                        masked_sentence = masked_sentence.replace(ent["word"], "[LOC]")
                masked_units.append(masked_sentence)
            else:
                masked_units.append(sentence)

        unit_embeddings = model.encode(masked_units)
        
        enriched = []
        for original_text, masked_text, emb in zip(units, masked_units, unit_embeddings):
            emb = np.array(emb, dtype=np.float32)
            similarities = [model.similarity(emb, label_vec) for label_vec in label_emb]
            sorted_similarities = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)

            top_matches = [
                {
                    "label": labels[idx],
                    "score": round(float(score), 4)
                }
                for idx, score in sorted_similarities[:3]
            ]

            enriched.append({
                "original": original_text,
                "masked": masked_text,
                "embedding": emb.tolist(),
                "top_labels": top_matches
            })

        item["sentence_analysis"] = enriched

    # Ensure output dir exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    write_jsonl(output_path, data)
    print(f"Processed {len(data)} entries. Output saved to {output_path}")


In [16]:
precisions = [0, 1, 2, 3, 4, 5]
prompt_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


for prec in precisions:
    for prompt_id in prompt_ids:
        input_path = f"prompt engineering/results/precision test/geo counts update/precision_{prec}/results_prompt_{prompt_id}.jsonl"
        output_path = f"prompt engineering/results/precision test/accuracy masked entities/precision_{prec}/predicted_labels_{prompt_id}.jsonl"
        emb_l_path = "Data/labels_with_embeddings.jsonl"

        process_and_label_sentences_m(input_path, emb_l_path, output_path, per_sentence=False)

Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy masked entities/precision_0/predicted_labels_1.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy masked entities/precision_0/predicted_labels_2.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy masked entities/precision_0/predicted_labels_3.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy masked entities/precision_0/predicted_labels_4.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy masked entities/precision_0/predicted_labels_5.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy masked entities/precision_0/predicted_labels_6.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy masked entities/precision_0/predicted_labels_7.jsonl

In [5]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
# model = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-3.2-3B-Instruct", token=access_token
# )
# print("done")

### Test #1 - from city to location

In [11]:
##### TEST 1 - FROM CITY TO LOCATION ######
model_name = "meta-llama/Llama-3.2-3B-Instruct"

f_file_path = "prompt engineering/results/demonstration_set.jsonl"

t_file_paths = ["prompt engineering/results/test 1 results/prompt1_llama.jsonl", 
               "prompt engineering/results/test 1 results/prompt2_llama.jsonl",
               "prompt engineering/results/test 1 results/prompt3_llama.jsonl",
               "prompt engineering/results/test 1 results/prompt4_llama.jsonl",
               "prompt engineering/results/test 1 results/prompt5_llama.jsonl",
               "prompt engineering/results/test 1 results/prompt6_llama.jsonl",
               "prompt engineering/results/test 1 results/prompt7_llama.jsonl",
               "prompt engineering/results/test 1 results/prompt8_llama.jsonl",
               "prompt engineering/results/test 1 results/prompt9_llama.jsonl"] # save generated coordinates\n",

r_file_path = "prompt engineering/responses/places_responses_llama.jsonl"

prompt_file_paths = ["prompt engineering/prompts task #1/prompt1.txt",
                     "prompt engineering/prompts task #1/prompt2.txt",
                     "prompt engineering/prompts task #1/prompt3.txt",
                     "prompt engineering/prompts task #1/prompt4.txt",
                     "prompt engineering/prompts task #1/prompt5.txt",
                     "prompt engineering/prompts task #1/prompt6.txt",
                     "prompt engineering/prompts task #1/prompt7.txt",
                     "prompt engineering/prompts task #1/prompt8.txt",
                     "prompt engineering/prompts task #1/prompt9.txt"]
                     
prompt_list = []
for p in prompt_file_paths:
    prompt_list.append(read_prompt(p))
print(prompt_list)


['I need the location data for {place} in decimal degrees — can you provide it?\n', 'What are the coordinates of the habitation {place}? Please return your answer in decimal degrees.', 'Give me the precise latitude and longitude of {place}, expressed in decimal degrees.\n', 'Can you provide the geographic coordinates for {place} in decimal format?\n', 'Please give me the precise location of {place} in decimal-degree format.\n', 'I need the exact GPS coordinates of {place} in decimal degree notation.\n', 'Could you return the coordinates (latitude and longitude) of {place} in decimal degrees?\n', 'Tell me the numerical latitude and longitude of {place} in decimal degrees.\n', 'Q: Can you provide the geographic coordinates for Tokyo in decimal format?       \nA: The latitude and longitude of Tokyo are 35.682839° N, 139.759455° E.\nQ: Can you provide the geographic coordinates for {place} in decimal format? \nA: \n']


In [7]:
#evaluate_multiple_prompts(prompt_list, f_file_path, t_file_paths, r_file_path, model_name, access_token, "prompt engineering/results/test 1 results/task_1_avg.jsonl")



_______________________________________
### Test \#2 - Now generation of descriptions

In [12]:
f_file_path_des = "prompt engineering/results/mmearth_coordinates.jsonl" # make data set

geo_file_path = "prompt engineering/results/geo_terms.txt"

task_2_eval = "prompt engineering/results/test 2 results/task_2_2_eval.jsonl"

t_file_paths_des = ["prompt engineering/results/test 2 results/prompt1_llama.jsonl", 
                   "prompt engineering/results/test 2 results/prompt2_llama.jsonl",
                   "prompt engineering/results/test 2 results/prompt3_llama.jsonl",
                   "prompt engineering/results/test 2 results/prompt4_llama.jsonl",
                   "prompt engineering/results/test 2 results/prompt5_llama.jsonl",
                   "prompt engineering/results/test 2 results/prompt6_llama.jsonl",
                   "prompt engineering/results/test 2 results/prompt7_llama.jsonl",
                   "prompt engineering/results/test 2 results/prompt8_llama.jsonl"] 

r_file_path = "prompt engineering/responses/places_responses_llama_test2_2.jsonl"

prompt_file_paths_des = ["prompt engineering/prompts task #2/prompt1.txt",
                         "prompt engineering/prompts task #2/prompt2.txt",
                         "prompt engineering/prompts task #2/prompt3.txt",
                         "prompt engineering/prompts task #2/prompt4.txt",
                         "prompt engineering/prompts task #2/prompt5.txt",
                         "prompt engineering/prompts task #2/prompt6.txt",
                         "prompt engineering/prompts task #2/prompt7.txt",
                         "prompt engineering/prompts task #2/prompt8.txt"]
prompt_list_des = []
for p in prompt_file_paths_des:
    prompt_list_des.append(read_prompt(p))
print(prompt_list_des)

# read coords
coordinates_mmearth = read_jsonl(f_file_path_des) # lat and lon

['Describe the natural landscape found at this location: {coords}. Include details about the terrain, vegetation, wildlife, climate and any notable geographical features.\n', 'Imagine you are standing at {coords}. What do you see?\n', 'Can you provide me with a description of the landscape found at {coords}?\n', 'Give a short but precise description of the natural world around {coords}. Please include details about the terrain, climate, and any notable natural elements.\n', 'What can be said about the landscape at {coords}? Describe the physical terrain, vegetation, wildlife, and climatic conditions.\n', 'What is the overall geographical setting at {coords}? Describe the land’s features, vegetation, wildlife, and climatic conditions in this area.\n', 'Describe the physical environment found at {coords}. Use geographical terms to characterize the terrain, vegetation, wildlife, and weather.\n', 'Q: What can be said about the landscape at {coords}? Describe the physical terrain, vegetatio

In [7]:
# Load geographical terms from a file
def load_geo_terms(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return [line.strip().lower() for line in file.readlines() if line.strip()]

# Read terms from the file
geo_terms = load_geo_terms("prompt engineering/results/geo_terms.txt")
                     
def count_geo_terms(description):
    description = description.lower()  # Convert to lowercase
    used_terms = [term for term in geo_terms if term in description]
    count = len(used_terms)
    
    return count, used_terms

# Example descriptions
descriptions = ["A large Mountains range with deep valleys and a fast-flowing RIver. There are snowy pastures",
    "Near a small town, located by a lake, surrounded by hills.",
    "A big area."]

# Count and score each description
for desc in descriptions:
    count, matches = count_geo_terms(desc)
    print(f"Description: {desc}")
    print(f"Geographical term count: {count}")
    print(f"Matched terms: {', '.join(matches) if matches else 'None'}\n")
    

Description: A large Mountains range with deep valleys and a fast-flowing RIver. There are snowy pastures
Geographical term count: 5
Matched terms: mountain, valley, river, pasture, snow

Description: Near a small town, located by a lake, surrounded by hills.
Geographical term count: 4
Matched terms: hill, lake, town, near

Description: A big area.
Geographical term count: 0
Matched terms: None



In [8]:
coords = read_jsonl(f_file_path_des)
print(coords[0]["latitude"])

prompt = insert_coordinates(prompt_list_des[0], coords[0])
print(prompt)

27.098393548966815
Describe the natural landscape found at this location: 27.098393548966815, -106.64053750001729. Include details about the terrain, vegetation, wildlife, climate and any notable geographical features.



## run test #2

In [9]:
import time
subset = coordinates_mmearth[16]

# Start the timer
start_time = time.time()

generate_description_for_all_coords(prompt_list_des, geo_file_path, f_file_path_des, t_file_paths, r_file_path, model_name, access_token, coordinates_mmearth[16], task_2_eval)

# End the timer
end_time = time.time()

# Print execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


prompt: Describe the natural landscape found at this location: 42.52311858775062, -6.833938499987374. Include details about the terrain, vegetation, wildlife, climate and any notable geographical features.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Description: The location 42.52311858775062, -6.833938499987374 corresponds to the city of Casablanca, Morocco. 

The terrain is generally flat, with some low hills and coastal plains. The city is situated on the Atlantic coast, with the Atlantic Ocean to the west. 

Vegetation is Mediterranean, with olive groves, citrus trees, and palm trees. 

Climate is subtropical, with mild winters and hot summers. 

Notable geographical features include the Atlantic coast, the Atlantic Ocean, and the nearby Atlas Mountains.
Geographical term count: 15
Matched terms: mountain, hill, ocean, city, west, near, tree, grove, vegetation, flat, coast, tropic, climate, hot, plain

prompt: Imagine you are standing at 42.52311858775062, -6.833938499987374. What do you see?



...

Description: The coordinates 42.52311858775062, -6.833938499987374 correspond to a location in the Atlantic Ocean, approximately 1,000 km off the coast of Morocco. 

The terrain is a seamount, a submerged volcanic island. The surrounding waters are characterized by a mix of continental shelf and abyssal plains. 

Vegetation is limited due to the marine environment. 

Wildlife includes various marine species such as tuna, swordfish, and sharks. 

Climatic conditions are subtropical, with mild temperatures ranging from 18-24°C (64-75°F) throughout the year.
Geographical term count: 8
Matched terms: ocean, sea, surrounding, vegetation, water, coast, tropic, plain

prompt: What is the overall geographical setting at 42.52311858775062, -6.833938499987374? Describe the land’s features, vegetation, wildlife, and climatic conditions in this area.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Description: The geographical setting at 42.52311858775062, -6.833938499987374 corresponds to the city of Dakar, Senegal, West Africa.

Dakar is situated on a coastal plain, bordering the Atlantic Ocean. The terrain is generally flat, with some low hills and dunes. The climate is tropical, with high temperatures and high humidity throughout the year.

Vegetation is mostly composed of tropical savannas, with acacia trees, baobabs, and mangroves along the coast. The region is also home to a variety of wildlife, including monkeys, antelopes, and birds.

The climate is characterized by a wet season from May to October and a dry season from November to April. Average temperatures range from 22°C to 30°C (72°F to 86°F).
Geographical term count: 16
Matched terms: hill, dune, ocean, sea, city, west, along, tree, grove, vegetation, flat, coast, savanna, tropic, climate, plain

prompt: Describe the physical environment found at 42.52311858775062, -6.833938499987374. Use geographical terms to cha

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Description: The coordinates 42.52311858775062, -6.833938499987374 correspond to a location in the Atlantic Ocean, approximately 1,000 km off the coast of Senegal, West Africa.

The terrain is a vast expanse of open ocean, with a depth of approximately 3,000 meters. The seafloor is characterized by a gentle slope, with a few underwater ridges and valleys.

Vegetation is limited to a few scattered coral reefs and seagrass beds, providing habitat for marine life such as sea turtles, rays, and sharks.

Weather patterns are influenced by the trade winds and the Intertropical Convergence Zone (ITCZ), resulting in moderate temperatures (22-28°C) and high humidity throughout the year.
Geographical term count: 12
Matched terms: valley, ridge, ocean, sea, west, vegetation, water, coast, grass, tropic, open, scattered

prompt: Q: What can be said about the landscape at 42.52311858775062, -6.833938499987374? Describe the physical terrain, vegetation, wildlife, and climatic conditions.
A: 

Descri

___________________________
## Sentence embedding

In [6]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

### Embedding of labels

In [ ]:
# embed_labels()

### Embedding of describtions

In [5]:
des = read_jsonl("prompt engineering/results/test 2 results/verbosity_test_run2_2dec/results_prompt_1.jsonl")
des_ex = des[0]['response']
print(des[0])

label_emb = read_jsonl("Data/labels_with_embeddings.jsonl")
# Extract label embeddings and labels
labels = [entry['label'] for entry in label_emb]
label_embeddings = np.array([entry['embedding'] for entry in label_emb])

{'location': 1, 'response': 'The location 37.35, -7.84 corresponds to the city of Marrakech, Morocco. \n\nMarrakech is situated in the Atlas Mountains, with a semi-arid climate. The terrain is characterized by rugged hills, valleys, and plateaus. \n\nVegetation is primarily composed of olive trees, palm groves, and thorny bushes. \n\nWildlife includes Barbary macaques, fennec foxes, and various bird species.\n\nNotable geographical features include the Atlas Mountains, the Ourika Valley, and the Koutoubia Mosque.', 'count': 11}


#### Create embeddings of generated text and find best matching label

In [8]:
precisions = [0, 1, 2, 3, 4, 5]
prompt_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for prec in precisions:
    for prompt_id in prompt_ids:
        input_path = f"prompt engineering/results/precision test/geo counts update/precision_{prec}/results_prompt_{prompt_id}.jsonl"
        output_path = f"prompt engineering/results/precision test/accuracy whole text update/precision_{prec}/predicted_labels_{prompt_id}.jsonl"
        emb_l_path = "Data/labels_with_embeddings.jsonl"

        process_and_label_sentences(input_path, emb_l_path, output_path, per_sentence=False)


Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy whole text update/precision_0/predicted_labels_1.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy whole text update/precision_0/predicted_labels_2.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy whole text update/precision_0/predicted_labels_3.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy whole text update/precision_0/predicted_labels_4.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy whole text update/precision_0/predicted_labels_5.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy whole text update/precision_0/predicted_labels_6.jsonl
Processed 100 entries. Output saved to prompt engineering/results/precision test/accuracy whole text update/precision_0/predicted_

_______________________________
### Evaluating the predicted labels

**True label distribution**

In [24]:
def print_sentences_with_predicted_label(jsonl_path, target_label):
    data = read_jsonl(jsonl_path)

    print(f"\nSentences with predicted label: '{target_label}'\n" + "-"*60)

    for entry in data:
        for sent_entry in entry.get("sentence_embeddings", []):
            top_preds = sent_entry.get("top_predictions", [])
            if top_preds:
                # Check if top-1 prediction is the target label
                if top_preds[0]["label"] == target_label:
                    sentence = sent_entry.get("sentence", "")
                    score = top_preds[0]["score"]
                    print(f"Score: {score} — {sentence}")


In [25]:
target = "Land principally occupied by agriculture, with significant areas of natural vegetation"
print_sentences_with_predicted_label("prompt engineering/results/test 2 results/best_prompt/all_done/descriptions_with_predicted_labels_2dec_0_update_1.jsonl", target)


Sentences with predicted label: 'Land principally occupied by agriculture, with significant areas of natural vegetation'
------------------------------------------------------------
Score: 0.4429 — The terrain is characterized by rugged hills, valleys, and plateaus.
Score: 0.351 — Notable geographical features include the Atlas Mountains, the Ourika Valley, and the Atlas foothills.
Score: 0.441 — Riga's landscape is characterized by a mix of urban and natural elements.
Score: 0.4262 — The terrain is generally flat, with some gentle slopes and low-lying areas.
Score: 0.6299 — Plant cover is limited in urban areas, but forests and parks are abundant in the surrounding countryside.
Score: 0.4088 — Wildlife in the area includes various bird species, such as seagulls, ducks, and geese.
Score: 0.3251 — The region experiences moderate precipitation, with an average annual rainfall of around 600 mm.
Score: 0.4099 — Wildlife in the area includes various bird species, such as seagulls, geese, a

In [21]:
from collections import Counter
target = "Land principally occupied by agriculture, with significant areas of natural vegetation"

veg_sentences = []
non_veg_sentences = []
data = read_jsonl("prompt engineering/results/test 2 results/best_prompt/all_done/descriptions_with_predicted_labels_2dec_0_update_1.jsonl")

for entry in data:
    for sent_entry in entry["sentence_embeddings"]:
        if "vegetation" in sent_entry["sentence"].lower() and (sent_entry["top_predictions"][0]["label"] == target or sent_entry["top_predictions"][1]["label"] == target or sent_entry["top_predictions"][2]["label"] == target):
            veg_sentences.append(sent_entry["sentence"])
        elif "vegetation" in sent_entry["sentence"].lower():
            non_veg_sentences.append(sent_entry["sentence"])
#print(veg_sentences)

print(f"{len(veg_sentences)} out of {len(non_veg_sentences) + len(veg_sentences)} sentences containing \"vegetation\" gets the label {target}")


5 out of 5 sentences containing "vegetation" gets the label Land principally occupied by agriculture, with significant areas of natural vegetation


In [22]:
veg_sentences = []
non_veg_sentences = []
correctly_predicted_veg_sentences = []

data = read_jsonl("prompt engineering/results/test 2 results/best_prompt/all_done/descriptions_with_predicted_labels_2dec_0_update_1.jsonl")

for entry in data:
    true_labels = entry.get("labels", [])
    for sent_entry in entry["sentence_embeddings"]:
        if "vegetation" in sent_entry["sentence"].lower():
            # Check if the target label is among the top predictions
            predicted_labels = [pred["label"] for pred in sent_entry["top_predictions"]]
            #print(predicted_labels)
            
            if target in predicted_labels:
                veg_sentences.append(sent_entry["sentence"])

                # Check if target is also a true label
                if target in true_labels:
                    correctly_predicted_veg_sentences.append(sent_entry["sentence"])
            else:
                non_veg_sentences.append(sent_entry["sentence"])

# Summary
total_with_vegetation = len(veg_sentences) + len(non_veg_sentences)
print(f'{len(veg_sentences)} out of {total_with_vegetation} sentences containing "vegetation" were assigned the label "{target}".')
print(f'{len(correctly_predicted_veg_sentences)} of those were correct predictions (i.e., the target was in the true labels).')


5 out of 5 sentences containing "vegetation" were assigned the label "Land principally occupied by agriculture, with significant areas of natural vegetation".
0 of those were correct predictions (i.e., the target was in the true labels).


_____________________
**Extracting dataset (BigEarthNet) from parquet** 

In [ ]:

# # Load the Parquet file
# metadata_df = pd.read_parquet("Data/metadata.parquet", engine="pyarrow")

# # Select only the first 30 rows
# metadata_df = metadata_df.head(30)

# # Convert to JSONL format
# with open("metadata_30.jsonl", "w") as jsonl_file:
#     for _, row in metadata_df.iterrows():
#         row_dict = row.to_dict()
        
#         # Convert NumPy arrays to Python lists
#         for key, value in row_dict.items():
#             if isinstance(value, np.ndarray):
#                 row_dict[key] = value.tolist()

#         jsonl_file.write(json.dumps(row_dict) + "\n")

# print("Conversion complete! Saved as metadata_30.jsonl")


### **BigEarthNet**

_______________
validation set

In [3]:
from torchgeo.datasets import BigEarthNet
import tarfile
from tqdm import tqdm

In [ ]:
dataset_train = BigEarthNet(
    root='BigEarthNet_data_train_s2',
    split='train',
    bands='s2',
    num_classes=19,
    transforms=None,
    download=True,
    checksum=False
)

ERROR! Session/line number was not unique in database. History logging moved to new session 79


100%|██████████████████████████████████████████████████████████████████████████████| 70.0G/70.0G [23:20<00:00, 50.0MB/s]


In [ ]:
import shutil
shutil.rmtree("BigEarthNet_data_train_s2/BigEarthNet-v1.0", ignore_errors=True)


In [7]:
from torchgeo.datasets import BigEarthNet

# Define the root directory where the dataset will be stored
root_train = "BigEarthNet_data_train_s2/BigEarthNet-v1.0"

# Initialize dataset
dataset_train = BigEarthNetS1Custom(root_train)

In [8]:
# Access a sample
image, metadata = dataset_train[0]
print(f"Image shape: {image.shape}")
print(f"Metadata: {metadata}")
print(f"length of validation set: {len(dataset_val)}")

BigEarthNet_data_train_s2/BigEarthNet-v1.0/S2A_MSIL2A_20171002T94032_61_82/S2A_MSIL2A_20171002T94032_61_82_labels_metadata.json


FileNotFoundError: [Errno 2] No such file or directory: 'BigEarthNet_data_train_s2/BigEarthNet-v1.0/S2A_MSIL2A_20171002T94032_61_82/S2A_MSIL2A_20171002T94032_61_82_VV.tif'

In [1]:
import os
import json
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torchvision.transforms.functional as TF

class BigEarthNetS2Custom(Dataset):
    def __init__(self, root, selected_bands=None, target_size=(120, 120), transform=None):
        self.root = root
        self.patch_folders = [
            f for f in os.listdir(root)
            if os.path.isdir(os.path.join(root, f)) and not f.startswith('.') and '.ipynb_checkpoints' not in f
        ]
        self.selected_bands = selected_bands or [
            'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B8A', 'B11', 'B12'
        ]
        self.transform = transform or transforms.ToTensor()
        self.target_size = target_size  # target HxW, e.g., (120, 120) for 10m bands

    def __len__(self):
        return len(self.patch_folders)

    def __getitem__(self, idx):
        patch_folder_name = self.patch_folders[idx]
        patch_folder = os.path.join(self.root, patch_folder_name)

        band_tensors = []
        for band in self.selected_bands:
            band_path = os.path.join(patch_folder, f'{patch_folder_name}_{band}.tif')
            band_image = Image.open(band_path)

            # Resize all bands to the target size
            band_image_resized = band_image.resize(self.target_size, Image.BILINEAR)

            band_tensor = self.transform(band_image_resized)
            band_tensors.append(band_tensor)

        image = torch.cat(band_tensors, dim=0)

        metadata_path = os.path.join(patch_folder, f'{patch_folder_name}_labels_metadata.json')
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)

        metadata_dict = {
            'labels': metadata.get('labels', []),
            'coordinates': metadata.get('coordinates', {}),
            'projection': metadata.get('projection', ''),
            'corresponding_s1_patch': metadata.get('corresponding_s1_patch', ''),
            'scene_source': metadata.get('scene_source', ''),
            'acquisition_time': metadata.get('acquisition_time', ''),
            'location_name': patch_folder_name  # <- Add this!
        }

        return image, metadata_dict


In [5]:
class BigEarthNetS2Custom(Dataset):
    def __init__(self, root, selected_bands=None, target_size=(120, 120), transform=None, folder_list=None):
        self.root = root
        all_folders = [
            f for f in os.listdir(root)
            if os.path.isdir(os.path.join(root, f)) and not f.startswith('.') and '.ipynb_checkpoints' not in f
        ]

        # Filter by provided list
        if folder_list is not None:
            self.patch_folders = [f for f in all_folders if f in folder_list]
        else:
            self.patch_folders = all_folders

        self.selected_bands = selected_bands or [
            'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B8A', 'B11', 'B12'
        ]
        self.transform = transform or transforms.ToTensor()
        self.target_size = target_size

    def __len__(self):
        return len(self.patch_folders)

    def __getitem__(self, idx):
        patch_folder_name = self.patch_folders[idx]
        patch_folder = os.path.join(self.root, patch_folder_name)

        band_tensors = []
        for band in self.selected_bands:
            band_path = os.path.join(patch_folder, f'{patch_folder_name}_{band}.tif')
            band_image = Image.open(band_path)
            band_image_resized = band_image.resize(self.target_size, Image.BILINEAR)
            band_tensor = self.transform(band_image_resized)
            band_tensors.append(band_tensor)

        image = torch.cat(band_tensors, dim=0)

        metadata_path = os.path.join(patch_folder, f'{patch_folder_name}_labels_metadata.json')
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)

        metadata_dict = {
            'labels': metadata.get('labels', []),
            'coordinates': metadata.get('coordinates', {}),
            'projection': metadata.get('projection', ''),
            'corresponding_s1_patch': metadata.get('corresponding_s1_patch', ''),
            'scene_source': metadata.get('scene_source', ''),
            'acquisition_time': metadata.get('acquisition_time', ''),
            'location_name': patch_folder_name
        }

        return image, metadata_dict

In [8]:
from torchgeo.datasets import BigEarthNet

# Define the root directory where the dataset will be stored
root_train = "BigEarthNet_data_train_s2/BigEarthNet-v1.0"

split_file = "/home/fhd511/Geollm_project/BigEarthNet_data_train_s2/BigEarthNet-v1.0/locations_part_aa"

with open(split_file, "r") as f:
    folder_list = [os.path.basename(line.strip()) for line in f if line.strip()]


print(folder_list[:5])

# Initialize dataset
dataset_train = BigEarthNetS2Custom(root_train, folder_list=folder_list)

# Access a sample
image, metadata = dataset_train[0]
print(f"Image shape: {image.shape}")
print(f"Metadata: {metadata}")
print(f"length of train set: {len(dataset_train)}")

['S2A_MSIL2A_20170613T101031_0_45', 'S2A_MSIL2A_20170613T101031_0_46', 'S2A_MSIL2A_20170613T101031_0_47', 'S2A_MSIL2A_20170613T101031_0_48', 'S2A_MSIL2A_20170613T101031_0_49']
Image shape: torch.Size([12, 120, 120])
Metadata: {'labels': ['Discontinuous urban fabric', 'Sport and leisure facilities', 'Pastures', 'Land principally occupied by agriculture, with significant areas of natural vegetation'], 'coordinates': {'ulx': 646800, 'uly': 5897640, 'lrx': 648000, 'lry': 5896440}, 'projection': 'PROJCS["WGS 84 / UTM zone 29N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-9],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],P

In [11]:
# Initialize dataset
dataset_train_test = BigEarthNetS2Custom(root_train, folder_list=folder_list[:5])

In [28]:
# Access a sample
image, metadata = dataset_train_test[0]
print(f"Image shape: {image.shape}")
print(f"Metadata: {metadata}")
print(f"length of train set: {len(dataset_train)}")
print(len(dataset_train_test))
coordinates = metadata.get('coordinates', {})
print(coordinates)
ulx, uly = coordinates.get('ulx'), coordinates.get('uly')
print(ulx, uly)
projection_str = metadata.get('projection', '')
print(projection_str)
utm_zone = extract_utm_zone(projection_str)
print(utm_zone)
location_name = metadata.get('location_name', f"loc_{0}")
print(location_name)

Image shape: torch.Size([12, 120, 120])
Metadata: {'labels': ['Sea and ocean'], 'coordinates': {'ulx': 499980, 'uly': 7044840, 'lrx': 501180, 'lry': 7043640}, 'projection': 'PROJCS["WGS 84 / UTM zone 34N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",21],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32634"]]', 'corresponding_s1_patch': '', 'scene_source': '', 'acquisition_time': '', 'location_name': 'S2A_MSIL2A_20170613T101031_0_46'}
length of train set: 147292
5
{'ulx': 499980, 'uly': 7

In [30]:
# Save metadata
save_samples_to_jsonl(dataset_train_test, "metadata_part_aa_test.jsonl")

[DONE] Saved 5 entries to metadata_part_aa_test.jsonl


In [34]:
run_single_prompt(
    prompt_template="Give a description of the general geographical setting at {coords}, paying attention to terrain, climate, flora and fauna, and other distinctive natural features.",
    data_path="metadata_part_aa_test.jsonl",
    n_loc=5,
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    token=access_token,
    output_file="descriptions_test.jsonl",
    precision=3
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generating batch 0–5...


OSError: [Errno 28] No space left on device

In [6]:
# # Define the path where the dataset is stored or will be downloaded
# root = "bigearthnet_data/BigEarthNet-S1-v1.0"

# # Initialize dataset
# dataset = BigEarthNetS1Custom(root)

# # Access a sample
# image, metadata = dataset[0]
# print(f"Image shape: {image.shape}")
# print(f"Metadata: {metadata}")

In [18]:
# Extracted UTM Zone (from the projection metadata)
utm_zone = extract_utm_zone(metadata['projection'])

# Convert to decimal degrees using the extracted UTM zone
lon, lat = utm_to_decimal(
    coordinates['ulx'], coordinates['uly'], coordinates['lrx'], coordinates['lly'], utm_zone
)

print(f"Middle of patch: ({lat}, {lon})")

Middle of patch: (53.566560850780064, -6.502355915676299)


#### Creating a subset of val

In [7]:
#save_samples_to_jsonl(dataset_val, 1000, "Data/subsets/ben_1000_val_up_ran.jsonl")

bigearthnet_data_val/BigEarthNet-S1-v1.0/S1B_IW_GRDH_1SDV_20180327T183408_29SNB_85_54/S1B_IW_GRDH_1SDV_20180327T183408_29SNB_85_54_labels_metadata.json
32629
bigearthnet_data_val/BigEarthNet-S1-v1.0/S1A_IW_GRDH_1SDV_20170903T153258_35VNL_77_8/S1A_IW_GRDH_1SDV_20170903T153258_35VNL_77_8_labels_metadata.json
32635
bigearthnet_data_val/BigEarthNet-S1-v1.0/S1A_IW_GRDH_1SDV_20170818T160607_34WFS_32_50/S1A_IW_GRDH_1SDV_20170818T160607_34WFS_32_50_labels_metadata.json
32634
bigearthnet_data_val/BigEarthNet-S1-v1.0/S1B_IW_GRDH_1SDV_20180524T153951_35VNH_12_32/S1B_IW_GRDH_1SDV_20180524T153951_35VNH_12_32_labels_metadata.json
32635
bigearthnet_data_val/BigEarthNet-S1-v1.0/S1A_IW_GRDH_1SDV_20180223T154131_34WFT_84_41/S1A_IW_GRDH_1SDV_20180223T154131_34WFT_84_41_labels_metadata.json
32634
bigearthnet_data_val/BigEarthNet-S1-v1.0/S1A_IW_GRDH_1SDV_20180421T165046_33TWN_74_2/S1A_IW_GRDH_1SDV_20180421T165046_33TWN_74_2_labels_metadata.json
32633
bigearthnet_data_val/BigEarthNet-S1-v1.0/S1B_IW_GRDH_1SD

In [12]:
# model_name = "meta-llama/Llama-3.2-3B-Instruct"
# prompt = read_prompt("prompt engineering/prompts task #2/prompt1.txt")
# data_path = "Data/ben_100.jsonl"
# geo_terms = load_geo_terms("prompt engineering/results/geo_terms.txt")
# eval_path = "prompt engineering/results/test 2 results/test_eval.jsonl"

# verbosity_test(prompt, geo_terms, data_path, 10, model_name, access_token, eval_path)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


###  first run

In [5]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"
n_loc = 100
data_path = "Data/ben_1000_val_up.jsonl"
geo_terms = load_geo_terms("prompt engineering/results/geo_terms.txt")
eval_dir = "prompt engineering/results/test 2 results/verbosity_test_3"
prompt_file_paths_des = ["prompt engineering/prompts task #2/prompt1.txt",
                         "prompt engineering/prompts task #2/prompt2.txt",
                         "prompt engineering/prompts task #2/prompt3.txt",
                         "prompt engineering/prompts task #2/prompt4.txt",
                         "prompt engineering/prompts task #2/prompt5.txt",
                         "prompt engineering/prompts task #2/prompt6.txt",
                         "prompt engineering/prompts task #2/prompt7.txt",
                         "prompt engineering/prompts task #2/prompt8.txt"]
prompt_list_des = []
for p in prompt_file_paths_des:
    prompt_list_des.append(read_prompt(p))

num_prompts = len(prompt_file_paths_des) 

In [6]:
# DONT RUN AGAIN UNLESS YOU WANT TO OVERWRITE!!
#verbosity_test_up(prompt_list_des, geo_terms, data_path, n_loc, model_name, access_token, eval_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing Prompt 1, Location 1: Describe the natural landscape found at this location: 37.97, -8.83. Include details about the terrain, vegetation, wildlife, climate and any notable geographical features.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing Prompt 1, Location 2: Describe the natural landscape found at this location: 64.68, 24.54. Include details about the terrain, vegetation, wildlife, climate and any notable geographical features.



...

Processing Prompt 8, Location 98: Q: What can be said about the landscape at 43.7, 21.83? Describe the physical terrain, vegetation, wildlife, and climatic conditions.
A: 



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing Prompt 8, Location 99: Q: What can be said about the landscape at 60.01, 21.11? Describe the physical terrain, vegetation, wildlife, and climatic conditions.
A: 



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing Prompt 8, Location 100: Q: What can be said about the landscape at 62.87, 30.98? Describe the physical terrain, vegetation, wildlife, and climatic conditions.
A: 

Results for Prompt 8 saved to prompt engineering/results/test 2 results/verbosity_test_3/results_prompt_8.jsonl.


### side-track: how much does the precision of lat, lon mean?
i use prompt 1, 2 and 4 for this experiment

In [5]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"
data_path = "Data/subsets/ben_1000_val_up_ran.jsonl"
geo_terms = load_geo_terms("prompt engineering/results/geo_terms.txt")

n_loc = 100
precisions = [0, 1, 2, 3, 4, 5]

base_eval_dir = "prompt engineering/results/precision test/geo counts update" # if you run again put result in geo counts folder!

prompt_file_paths_des = ["prompt engineering/prompts task #2.0/prompt1.txt",
                         "prompt engineering/prompts task #2.0/prompt2.txt",
                         "prompt engineering/prompts task #2.0/prompt3.txt",
                         "prompt engineering/prompts task #2.0/prompt4.txt",
                         "prompt engineering/prompts task #2.0/prompt5.txt",
                         "prompt engineering/prompts task #2.0/prompt6.txt",
                         "prompt engineering/prompts task #2.0/prompt7.txt",
                         "prompt engineering/prompts task #2.0/prompt8.txt",
                         "prompt engineering/prompts task #2.0/prompt9.txt",
                         "prompt engineering/prompts task #2.0/prompt10.txt",
                         "prompt engineering/prompts task #2.0/prompt11.txt",
                         "prompt engineering/prompts task #2.0/prompt12.txt"]

prompt_list_des = []
for p in prompt_file_paths_des:
    prompt_list_des.append(read_prompt(p))

num_prompts = len(prompt_file_paths_des) 
print(prompt_list_des)

['Describe the natural landscape found at this location: {coords}. Include details about the terrain, vegetation, wildlife, climate and any notable geographical features.\n', 'Give a short but precise of the natural world around {coords}. Please include details about the terrain, climate, and any notable natural elements.\n', 'What is the overall geographical setting at {coords}? Describe the land’s features, vegetation, wildlife, and climatic conditions in this area.\n', 'Describe the natural environment at this location: {coords}. Focus on terrain, plant life, animal presence, climate, and notable geographical features.\n', 'At the coordinates {coords}, provide a detailed description of the natural landscape, including the topography, vegetation, wildlife, climate, and any significant physical features.\n', 'Give a concise yet detailed overview of the natural environment around {coords}, mentioning the terrain, climate, and any remarkable natural elements.\n', 'Provide a short but ac

In [6]:
# DONT RUN AGAIN UNLESS YOU WANT TO OVERWRITE!!
for precision in precisions:
    print(f"\n=== Running for precision {precision} ===")

    # Create subdirectory per precision
    eval_dir = os.path.join(base_eval_dir, f"precision_{precision}")
    
    verbosity_test_up(
        prompts=prompt_list_des,
        geo_terms=geo_terms,
        data_path=data_path,
        n_loc=n_loc,
        model_name=model_name,
        token=access_token,
        eval_dir=eval_dir,
        precision=precision
    )


=== Running for precision 0 ===


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



 Processing Prompt Template 1...
Generating 100 responses in batch...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Results saved to prompt engineering/results/precision test/geo counts update/precision_5/results_prompt_9.jsonl

 Processing Prompt Template 10...
Generating 100 responses in batch...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Results saved to prompt engineering/results/precision test/geo counts update/precision_5/results_prompt_10.jsonl

 Processing Prompt Template 11...
Generating 100 responses in batch...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Results saved to prompt engineering/results/precision test/geo counts update/precision_5/results_prompt_11.jsonl

 Processing Prompt Template 12...
Generating 100 responses in batch...
Results saved to prompt engineering/results/precision test/geo counts update/precision_5/results_prompt_12.jsonl


### second run

In [14]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"
n_loc = 100
data_path = "Data/subsets/ben_1000_val_up_ran.jsonl"
geo_terms = load_geo_terms("prompt engineering/results/geo_terms.txt")
eval_dir_2 = "prompt engineering/results/test 2 results/verbosity_test_run2_2dec"

prompt_file_paths_des_2 = ["prompt engineering/prompts task #2.0/prompt1.txt",
                           "prompt engineering/prompts task #2.0/prompt2.txt",
                           "prompt engineering/prompts task #2.0/prompt3.txt",
                           "prompt engineering/prompts task #2.0/prompt4.txt",
                           "prompt engineering/prompts task #2.0/prompt5.txt",
                           "prompt engineering/prompts task #2.0/prompt6.txt",
                           "prompt engineering/prompts task #2.0/prompt7.txt",
                           "prompt engineering/prompts task #2.0/prompt8.txt",
                           "prompt engineering/prompts task #2.0/prompt9.txt",
                           "prompt engineering/prompts task #2.0/prompt10.txt",
                           "prompt engineering/prompts task #2.0/prompt11.txt"]
prompt_list_des_2 = []
for p in prompt_file_paths_des_2:
    prompt_list_des_2.append(read_prompt(p))

num_prompts = len(prompt_file_paths_des_2) 
print(num_prompts)
#print(prompt_list_des_2)

11


In [ ]:
# DONT RUN AGAIN UNLESS YOU WANT TO OVERWRITE!!
verbosity_test_up(prompt_list_des_2, geo_terms, data_path, n_loc, model_name, access_token, eval_dir_2)

### Correctness test

#### 1000 loc using best prompt

In [45]:
best_prompt = "prompt engineering/prompts task #2.0/prompt1_update.txt"
model_name = "meta-llama/Llama-3.2-3B-Instruct"
n_loc = 1000
data_path = "Data/subsets/ben_1000_val_up_ran.jsonl"
eval_dir_best = "prompt engineering/results/test 2 results/best_prompt"

prompt_list_best = [(read_prompt(best_prompt))]
print(prompt_list_best)

['Describe the natural landscape found at this location: {coords}. Include details about the terrain, plant cover, wildlife, climate and any notable geographical features.\n']


In [46]:
read_jsonl(data_path)[0]['labels']

['Land principally occupied by agriculture, with significant areas of natural vegetation',
 'Sclerophyllous vegetation',
 'Transitional woodland/shrub']

In [ ]:
generate_describtions(prompt_list_best, data_path, n_loc, model_name, access_token, eval_dir_best, "update_1_1000")